In [ ]:
!pip install pymupdf
!pip install gdown

!pip install transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import fitz  #PyMuPDF
import pandas as pd
from tqdm import tqdm
from transformers import pipeline
import torch
import re
import random
random.seed(42)

In [ ]:
# output_path = "/content/drive/My Drive/all_resume_data.csv"
# if os.path.exists(output_path):
#   os.remove(output_path)

In [ ]:
# output_path = "/content/drive/My Drive/dataframe.csv"

# if os.path.exists(output_path):
#     os.remove(output_path)

In [ ]:
def extract_batch_texts(pdf_paths):
    rows = []
    for filepath in tqdm(pdf_paths):
        try:
            text = ""
            with fitz.open(filepath) as doc:
                for page in doc:
                    text += page.get_text()

            # Obtener categoría (nombre de subcarpeta)
            category = os.path.basename(os.path.dirname(filepath))
            filename = os.path.basename(filepath)
            rows.append({"filename": filename, "category": category, "resume_text": text})
        except Exception as e:
            print(f"Error with {filepath}: {e}")
    return pd.DataFrame(rows)


In [ ]:
zip_path = "/content/archive.zip"

if not os.path.exists(zip_path):
  !gdown --id 1rRT0wOg4pRguuZ3U8zygYftKdP55OmXH --output /content/
  # Descomprimir el ZIP
  !unzip -q /content/archive.zip -d /content/

In [ ]:
# Verificar estructura
!ls /content/data/data | head -n 10


ACCOUNTANT
ADVOCATE
AGRICULTURE
APPAREL
ARTS
AUTOMOBILE
AVIATION
BANKING
BPO
BUSINESS-DEVELOPMENT


# Lectura de los pdfs

In [ ]:
from glob import glob

pdf_folder = "/content/data/data"
pdf_files = sorted(glob(os.path.join(pdf_folder, "*/*.pdf")))
print(f"Total PDFs found: {len(pdf_files)}")

Total PDFs found: 2484


# Extraer texto de los pdfs

In [ ]:
batch_size = 50
output_path = "/content/drive/My Drive/Resume Dataset/all_resume_data.csv"
first_batch = True

number_files = 100;

pdf_files_random = random.sample(pdf_files, number_files)

#for i in range(0, len(pdf_files), batch_size):
for i in range(0, len(pdf_files_random), batch_size):
    batch_files = pdf_files_random[i:i + batch_size]
    print(f"📚 Processing batch {i // batch_size + 1} → Files {i} to {i + len(batch_files) - 1}")

    batch_df = extract_batch_texts(batch_files)

    # Guardar append
    batch_df.to_csv(output_path, mode='a', index=False, header=first_batch)
    first_batch = False

📚 Processing batch 1 → Files 0 to 49


100%|██████████| 50/50 [00:00<00:00, 96.19it/s]


📚 Processing batch 2 → Files 50 to 99


100%|██████████| 50/50 [00:00<00:00, 96.94it/s]


## Cargar csv generado y visualizar pirmeras filas

In [ ]:
# Ruta al CSV generado previamente
csv_path = '/content/drive/My Drive/Resume Dataset/all_resume_data.csv'

# Cargar el DataFrame
data = pd.read_csv(csv_path)

# Visualizar las primeras filas
data.head()

,filename,category,resume_text
0,28325193.pdf,ARTS,THEATER ARTS TEACHER & PROGRAM DIRECTOR\nSumma...
1,42487883.pdf,ACCOUNTANT,"ACCOUNTANT\nSummary\nA detail oriented, effici..."
2,56525735.pdf,CONSTRUCTION,SENIOR CONSTRUCTION PROJECT MANAGER\nSummary\n...
3,35468363.pdf,CHEF,SUSHI CHEF\nExecutive
4,10889157.pdf,CHEF,LEAD COOK (COOK III)\nSummary\nCharles Hundley...


## Cargar Modelo BART y Tokenizer

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Cargar tokenizer y modelo BART preentrenado para resumen
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Generación de resumenes con Bart


In [ ]:

# modo evaluación
model.eval()

data = data.sample(75).reset_index(drop=True) # 10 filas aleatorias

# Generar resúmenes
generated_summaries = []

for text in tqdm(data['resume_text'], desc="Generating summaries (without fine tuning)"):
    # Tokenizar entrada
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)

    # Generar resumen
    summary_ids = model.generate(
        inputs,
        max_length=300,
        min_length=40,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    # Decodificar y guardar resumen
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_summaries.append(summary)

#Agregar columna al DataFrame
data['generated_summary'] = generated_summaries


Generating summaries (without fine tuning): 100%|██████████| 75/75 [40:53<00:00, 32.71s/it]


In [ ]:
data.to_csv("/content/drive/My Drive/Resume Dataset/dataframe.csv", index=False)

## Preparación dataset para entrenamiento

In [ ]:
data = pd.read_csv("/content/drive/My Drive/Resume Dataset/dataframe.csv")

# Filtrar datos válidos (por si hay vacíos)
train_data = data[
    data['resume_text'].notnull() &
    data['generated_summary'].notnull()
].reset_index(drop=True)

# Mostrar cuántos datos quedaron
print(f"Total training examples: {len(train_data)}")


Total training examples: 75


## Tokenizacion para posterior fine tuning
input: resume text

output (labels):generated summary

attention mask: generado a partir del modelo Bart sin fine tuning

In [ ]:

# Cargar tokenizer preentrenado de BART
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Tokenizar textos de entrada (resumes)
inputs = tokenizer(
    list(train_data["resume_text"]),
    max_length=1024,
    truncation=True,
    padding='max_length',
    return_tensors="pt"
)

# Tokenizar resúmenes objetivo
outputs = tokenizer(
    list(train_data["generated_summary"]),
    max_length=300,
    truncation=True,
    padding='max_length',
    return_tensors="pt"
)

# Extraer tensores
input_ids = inputs.input_ids
attention_mask = inputs.attention_mask
labels = outputs.input_ids

# Reemplazar padding tokens en labels por -100 para que no afecten el cálculo del loss
labels[labels == tokenizer.pad_token_id] = -100


## Dataset personalizado con pytorch:
Agrupa los tensores para posterior/ alimentar al modelo

In [ ]:
import torch
from torch.utils.data import Dataset

class ResumeSummaryDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return self.input_ids.size(0)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }


## Dataloader para entrenamiento

In [ ]:
from torch.utils.data import DataLoader

# Instanciar dataset
dataset = ResumeSummaryDataset(input_ids, attention_mask, labels)

# Crear DataLoader con batch de 2
train_dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


## Modelo de optimizacion y entrenamiento

In [ ]:
from transformers import BartForConditionalGeneration
from torch.optim import AdamW
from tqdm import tqdm

#modelo preentrenado
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("Usando:", device)

# Optimizador
optimizer = AdamW(model.parameters(), lr=1e-5)

# Entrenamiento
model.train()
for epoch in range(3):  # no epocas
    loop = tqdm(train_dataloader, leave=True)
    for batch in loop:
        # Mover datos al dispositivo
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Adelante (forward pass)
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask, #genera el valor agregado a la prediccion
            labels=labels
        )
        loss = outputs.loss

        # Atrás (backward pass)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Mostrar progreso
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())

save_path = "/content/drive/My Drive/Resume Dataset/modelo_finetuned"

# Guardar el modelo y el tokenizer fine-tuned
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


NameError: name 'torch' is not defined